# <font color='blue'>Data Science Challenge @ ITA 2022</font>
# <font color='blue'>Equipe DIOMGIS</font>

## <font color='blue'>Fase 1</font>

### <font color='blue'>TEMA DO DESAFIO</font>

![title](..\data\image\logo.jpeg)

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
#!pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
# !pip install -q -U watermark

In [ ]:
# Bibliotecas e Frameworks

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from time import time
from datetime import datetime
import os
import joblib

In [ ]:
# Versões dos pacotes usados neste jupyter notebook

%reload_ext watermark
%watermark -a "Equipe DIOMGIS" --iversions

In [ ]:
sns.set_style('whitegrid')
%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
verbose = 1
seed = 25

np.random.seed(seed)

In [ ]:
# Gerador dados sintéticos

size_sample = 200000

# Dados de Treino
# x
x1 = 10 * np.random.random(size_sample)
x2 = 10 * np.random.random(size_sample)
x3 = 10 * np.random.random(size_sample)
x4 = 10 * np.random.random(size_sample)
x5 = 10 * np.random.random(size_sample)
x6 = 10 * np.random.random(size_sample)
x7 = 10 * np.random.random(size_sample)

x_treino = np.dstack((x1, x2, x3, x4, x5, x6, x7))[0]

# y
y_treino = 3*(x1**(1/2)) + 2*(x2**2) + 4*x3 - 5*(x4**(3/2)) + x5 + x6**(3) - x7

# Dados de Validação
# x
x1 = 10 * np.random.random(int(0.1 * size_sample))
x2 = 10 * np.random.random(int(0.1 * size_sample))
x3 = 10 * np.random.random(int(0.1 * size_sample))
x4 = 10 * np.random.random(int(0.1 * size_sample))
x5 = 10 * np.random.random(int(0.1 * size_sample))
x6 = 10 * np.random.random(int(0.1 * size_sample))
x7 = 10 * np.random.random(int(0.1 * size_sample))

x_teste = np.dstack((x1, x2, x3, x4, x5, x6, x7))[0]

# y
y_teste = 3*(x1**(1/2)) + 2*(x2**2) + 4*x3 - 5*(x4**(3/2)) + x5 + x6**(3) - x7

In [ ]:
# Parametros fixos de treinamento

nKFold = 5
nPCA = 7

In [ ]:
# Principal Component Analysis
'''
reduceDim = PCA(n_components = nPCA,
                copy = True,
                whiten = False,
                svd_solver = 'auto',
                tol = 0.0,
                iterated_power = 'auto',
                n_oversamples = 10,
                power_iteration_normalizer = 'auto',
                random_state = None)

reduceDim.fit(x_treino)

x_treino = reduceDim.transform(x_treino)
x_teste = reduceDim.transform(x_teste)
'''

In [ ]:
# Definição dos parametros (GridSearch)

params_grid = {
               # ['squared_error', 'absolute_error', 'huber', 'quantile']
              'loss': ['squared_error'],
    
               # trade-off entre learning_rate e n_estimators
              'learning_rate': [0.1],
    
               # O número de estágios de reforço a serem executados 
              'n_estimators': [100],
    
               # A fração de amostras a ser usada para ajustar os alunos de base individuais
              'subsample': [1.0],
               
               # ['friedman_mse', 'squared_error', 'mse'] 
              'criterion': ['friedman_mse'],
    
               # O número mínimo de amostras necessárias para dividir um nó interno
              'min_samples_split': [2],
    
               # O número mínimo de amostras necessárias para estar em um nó folha
              'min_samples_leaf': [1],
    
               # A fração ponderada mínima da soma total de pesos necessária para estar em um nó folha
              'min_weight_fraction_leaf': [0.0],
    
               # A profundidade máxima dos estimadores de regressão individuais
              'max_depth': [3],
    
               # Um nó será dividido se esta divisão induzir uma diminuição da impureza maior ou igual a este valor
              'min_impurity_decrease': [0.0],
    
               # Um objeto estimador que é usado para calcular as previsões iniciais
              'init': [None],
    
              # ['auto', 'sqrt', 'log2'], int ou float. O número de recursos a serem considerados ao procurar a melhor divisão
              'max_features': [None],
    
              # O alfa-quantil da função de perda de huber e a função de perda de quantil.
              # Somente se loss='huber'ou loss='quantile'. Entre 0 e 1
              'alpha': [0.9],
    
              # [int]. Cultive árvores max_leaf_nodes da melhor maneira possível
              'max_leaf_nodes': [None],
              
              # Quando definido como True, reutilize a solução da chamada anterior para ajustar e adicione
              # mais estimadores ao ensemble, caso contrário, apenas apague a solução anterior
              'warm_start': [False],
    
              # A proporção de dados de treinamento a serem reservados como validação
              # definida para interrupção antecipada
              'validation_fraction': [0.1],
    
              # usado para decidir se a interrupção antecipada será usada para encerrar
              # o treinamento quando a pontuação de validação não estiver melhorando
              'n_iter_no_change': [None],
    
              # Tolerância para a parada antecipada
              'tol': [0.0001],
    
              # Parâmetro de complexidade usado para remoção de complexidade de custo mínimo.
              # A subárvore com a maior complexidade de custo que for menor do que ccp_alphaserá escolhida
              'ccp_alpha': [0.0]
}

In [ ]:
# Grid Search e Cross Validation

model = GradientBoostingRegressor(random_state = seed, verbose = verbose)

grid = GridSearchCV(estimator = model,
                    param_grid = params_grid,
                    scoring = 'neg_mean_squared_error',
                    refit = True,
                    cv = nKFold,
                    return_train_score = False,
                    # n_jobs = -2, # "-2": mantem 1 processador livre
                    # pre_dispatch = '2*n_jobs',
                    verbose = verbose)

In [ ]:
# View grid

grid

In [ ]:
# Treinamento

fit_params = {'monitor': None}

grid_result = grid.fit(x_treino, y_treino, **fit_params)

In [ ]:
# Resultado do SearchGridCV

pd.concat([
           pd.DataFrame(grid.cv_results_)[['rank_test_score', 'mean_test_score', 'mean_fit_time']],
           pd.DataFrame(grid.cv_results_['params'])
          ],
           axis=1,
           join='inner').set_index('rank_test_score').sort_values('rank_test_score')

In [ ]:
best_params = grid.best_params_
best_model = grid.best_estimator_

In [ ]:
# Coeficiente de Determinação R2

best_model.score(x_treino, y_treino)

In [ ]:
# Coeficiente de Determinação R2

best_model.score(x_teste, y_teste)

In [ ]:
joblib.dump(best_model, "bestModel/{}xgboostModel.pkl".format(time()))


### Recuperando Modelo Salvo

In [ ]:
timeSaved = 1665772704.9439006
recoveryModel = joblib.load("bestModel/{}xgboostModel.pkl".format(timeSaved))

In [ ]:
recoveryModel.score(x_treino, y_treino)

In [ ]:
recoveryModel.score(x_teste, y_teste)

In [ ]:
mseTreino = mean_squared_error(y_treino, recoveryModel.predict(x_treino))
print("The mean squared error (MSE) on train set: {:.4f}".format(mseTreino))

In [ ]:
mseTeste = mean_squared_error(y_teste, recoveryModel.predict(x_teste))
print("The mean squared error (MSE) on train set: {:.4f}".format(mseTeste))

## Carregando o Conjunto de dados

## Análise Exploratória de Dados

### Análise n - XXX

## Pré-Processamento de Dados Para Construção de Modelos de Machine Learning

### Padronização

###  Construção, Treinamento e Avaliação do Modelo 1 com Regressão Linear (Benchmark)

### Avaliação do Modelo

### Métricas

### Resíduos

###  Construção, Treinamento e Avaliação do Modelo n com XXX

## Seleção do Modelo

## Conclusão

# Fim